In [2]:
import dgl
import math
import numpy as np
np.seterr(all="ignore")
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
import sys
import socket
import time
import random
import glob
import argparse, json
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader

from common import load_weights
from load_data import Graph_DataLoader
from wholemodel_pretrain import Net
from index2 import get_reactant_batch, get_reagent_batch, get_product_batch
from transformer import make_model

In [2]:

use_gpu = False
if torch.cuda.is_available() and use_gpu:
    print('cuda available with GPU:',torch.cuda.get_device_name(0))
    device = torch.device("cuda")
else:
    print('cuda not available')
    device = torch.device("cpu")
   

cuda not available


In [3]:
#load default config
with open('/fs/ess/PCON0041/xiaohu/MAT/Configs/w_best.json') as f:
    config = json.load(f)
# network parameters
net_params = config['net_params']
net_params['device'] = device
net_params['eb_n_layers'] = int(net_params['eb_n_layers'])
 
net_params['self_n_layers'] = int(net_params['self_n_layers'])
net_params['self_num_heads'] = int(net_params['self_num_heads'])
net_params['self_input_dim'] = int(net_params['self_input_dim'])
net_params['self_hidden_dim'] = int(net_params['self_hidden_dim'])
net_params['self_out_dim'] = int(net_params['self_out_dim'])  
net_params['edge_dim'] = int(net_params['edge_dim']) 
net_params['cross_num_heads'] = int(net_params['cross_num_heads'])
net_params['cross_input_dim'] = int(net_params['cross_input_dim'])
net_params['cross_hidden_dim'] = int(net_params['cross_hidden_dim'])
net_params['cross_out_dim'] = int(net_params['cross_out_dim'])  
net_params['dropout'] = float(net_params['dropout'])  
net_params['molecule_pooling_method'] = str(net_params['molecule_pooling_method'])
net_params['residual'] = True if net_params['residual']==True else False
net_params['readout'] = net_params['readout']
net_params['layer_norm'] = True if net_params['layer_norm']==True else False
net_params['batch_norm'] = True if net_params['batch_norm']==True else False
print(net_params)

params = config['params']
params['use_pretrain'] = bool(True) if params['use_pretrain']=='True' else bool(False)
print(params)

{'eb_n_layers': 8, 'self_n_layers': 5, 'self_num_heads': 16, 'self_input_dim': 1024, 'self_hidden_dim': 256, 'self_out_dim': 256, 'edge_dim': 7, 'cross_num_heads': 16, 'cross_input_dim': 256, 'cross_hidden_dim': 256, 'cross_out_dim': 256, 'dropout': 0.0, 'molecule_pooling_method': 'con', 'residual': False, 'readout': 'False', 'layer_norm': False, 'batch_norm': False, 'device': device(type='cpu')}
{'use_pretrain': True}


# creating our model using pretrain weights

In [4]:
embedding_transformer_params = {
            'd_atom': 28, #28 fixed
            'd_model': 1024,  #1024 fixed
            'N': 8, #8 tunnale
            'h': 16, #16 fixed
            'N_dense': 1,
            'lambda_attention': 0.33, #0.33 fixed
            'lambda_distance': 0.33, #0.33 fixed
            'leaky_relu_slope': 0.1, #fixed
            'dense_output_nonlinearity': 'relu', 
            'distance_matrix_kernel': 'exp', 
            'dropout': 0,
            'aggregation_type': 'mean'
        }
eb_model = make_model(**embedding_transformer_params).to(device)

print('============Loading pretrained weights to generate initialization============')
load_weights(eb_model, 8)
for name, param_i in eb_model.named_parameters():
    param_i.requires_grad = False



print('============Creating new layers============')
transformer_params = {
            'd_atom': net_params['self_input_dim'], #1024 fixed
            'd_model': 256, #tunnable
            'N': net_params['self_n_layers'], #8 tunnale
            'h': 16, #16 fixed
            'N_dense': 1,
            'lambda_attention': 0.33, #0.33 fixed
            'lambda_distance': 0.33, #0.33 fixed
            'leaky_relu_slope': 0.1, #fixed
            'dense_output_nonlinearity': 'relu', 
            'distance_matrix_kernel': 'exp', 
            'dropout': 0, #0.0 
            'aggregation_type': 'mean'
        }
model_r = make_model(**transformer_params).to(device)
model_a = make_model(**transformer_params).to(device)
model_p = make_model(**transformer_params).to(device)

print('============Creating Model============')
model = Net(net_params, model_r, model_a, model_p)
model = model.to(device)

print('============Loading weights============')
best = torch.load('/fs/ess/PCON0041/xiaohu/MAT/results/temp/models/best.pkl', map_location=torch.device('cpu'))
model_state_dict = model.state_dict()
for name, param in best.items():
    print(name)
    print(param)
    model_state_dict[name] = param


============Loading pretrained weights to generate initialization============
============Creating new layers============
============Creating Model============
============Loading weights============
transformer_r.encoder.layers.0.self_attn.linears.0.weight
tensor([[-0.0953, -0.0148,  0.0634,  ..., -0.0079,  0.0183, -0.0860],
        [ 0.1053,  0.1036,  0.0029,  ..., -0.0343,  0.0560,  0.0953],
        [-0.0074, -0.0196,  0.0150,  ...,  0.0324,  0.0025,  0.0241],
        ...,
        [-0.0985, -0.0059, -0.0011,  ...,  0.0263, -0.0971,  0.1192],
        [ 0.0190,  0.0560, -0.0457,  ..., -0.0722,  0.0850, -0.0173],
        [-0.0197, -0.0319, -0.0012,  ..., -0.0713, -0.0715, -0.0045]])
transformer_r.encoder.layers.0.self_attn.linears.0.bias
tensor([ 3.5071e-02,  4.7894e-02,  5.1775e-02, -1.4330e-02, -1.5666e-02,
        -1.9476e-02,  3.1317e-02,  6.7561e-02,  1.6854e-03, -2.4535e-02,
         1.5368e-02,  8.6640e-02, -1.7773e-02, -5.4429e-04,  4.5447e-02,
        -4.5520e-02,  4.4900e-02

In [5]:
#loading data
data_dir = '/fs/ess/PCON0041/xiaohu/MAT/Data/preprocessed_datasets/BH/BH_processed.csv'
data = pd.read_csv(data_dir) #reactions data
with open('/fs/ess/PCON0041/xiaohu/MAT/Data/preprocessed_datasets/BH/BH_reactant_feats.pkl', 'rb') as f:
    r_feats = pickle.load(f)
with open('/fs/ess/PCON0041/xiaohu/MAT/Data/preprocessed_datasets/BH/BH_reagents_feats.pkl', 'rb') as f:
    a_feats = pickle.load(f)
with open('/fs/ess/PCON0041/xiaohu/MAT/Data/preprocessed_datasets/BH/BH_products_feats.pkl', 'rb') as f:
    p_feats = pickle.load(f)

result_table = torch.zeros(data.shape[0],2)

In [6]:
def evaluate_network_for_analysis(eb_model, model, device, data_loader, params, r_feats, a_feats, p_feats, eval_index, result_table):
    eb_model.eval()
    model.eval()
    training = bool(False)
    with torch.no_grad():
        for iter, (batch_index, batch_targets) in enumerate(data_loader):
            if iter%500 == 0:
                print(iter)
            batch_targets = batch_targets.float().to(device)
            index_indeed = torch.Tensor(eval_index[batch_index])
            batch_targets_sorted = batch_targets[torch.sort(index_indeed)[1]].float().to(device)
            
            r_batch_adjacency_matrix, r_batch_node_features, r_batch_distance_matrix, r_batch_mask, r_batch_center_index, r_batch_index  = get_reactant_batch(eb_model, r_feats, eval_index, batch_index, params, device)
            print(r_batch_node_features.shape)
            a_batch_adjacency_matrix, a_batch_node_features, a_batch_distance_matrix, a_batch_mask, a_batch_index = get_reagent_batch(eb_model, a_feats, eval_index, batch_index, params, device)
            p_batch_adjacency_matrix, p_batch_node_features, p_batch_distance_matrix, p_batch_mask, p_batch_index = get_product_batch(eb_model, p_feats, eval_index, batch_index, params, device)

            batch_scores, batch_representation, r_h, a_h, p_h = model.forward(r_batch_adjacency_matrix, r_batch_node_features, r_batch_distance_matrix, r_batch_mask, r_batch_center_index, r_batch_index, 
            a_batch_adjacency_matrix, a_batch_node_features, a_batch_distance_matrix, a_batch_mask, a_batch_index, 
            p_batch_adjacency_matrix, p_batch_node_features, p_batch_distance_matrix, p_batch_mask, p_batch_index, training, device) 

            batch_scores = batch_scores.float()
            result_table[torch.sort(index_indeed)[0],0] = batch_scores.items()
            result_table[torch.sort(index_indeed)[0],1] = batch_targets_sorted.items()

    return result_table

In [7]:
split_set_num = 1
with open('/fs/ess/PCON0041/xiaohu/MAT/Data/preprocessed_datasets/BH/train_test_idxs.pickle', 'rb') as handle:
    idx_dict = pickle.load(handle)
            
train_index = idx_dict['train_idx'][split_set_num]
test_index = idx_dict['test_idx'][split_set_num]
valid_index = train_index[int(0.9*len(train_index)):]
train_index = train_index[0:int(0.9*len(train_index))]

training_set = Graph_DataLoader(data.loc[train_index].values, data_dir)
validation_set = Graph_DataLoader(data.loc[valid_index].values, data_dir) 
testing_set = Graph_DataLoader(data.loc[test_index].values, data_dir)

train_loader = DataLoader(training_set, batch_size=32, shuffle=True)
valid_loader = DataLoader(validation_set, batch_size=32, shuffle=False)
test_loader = DataLoader(testing_set, batch_size=32, shuffle=False)
print("Training Graphs Batches: ", len(train_loader))
print("Valid Graphs Batches: ", len(valid_loader))
print("Test Graphs Batches: ", len(test_loader))

result_table = evaluate_network_for_analysis(eb_model, model, device, train_loader, params, r_feats, a_feats, p_feats, train_index, result_table) 
print(result_table)
result_table = evaluate_network_for_analysis(eb_model, model, device, valid_loader, params, r_feats, a_feats, p_feats, valid_index, result_table) 
result_table = evaluate_network_for_analysis(eb_model, model, device, test_loader, params, r_feats, a_feats, p_feats, test_index, result_table) 

Training Graphs Batches:  78
Valid Graphs Batches:  9
Test Graphs Batches:  38
0
torch.Size([64, 12, 1024])


AttributeError: 'Tensor' object has no attribute 'items'

In [7]:
x = torch.randn(1,4)
print(x)
torch.sort(x)

tensor([[-0.7916, -0.4288, -1.2943,  1.8502]])


torch.return_types.sort(
values=tensor([[-1.2943, -0.7916, -0.4288,  1.8502]]),
indices=tensor([[2, 0, 1, 3]]))

In [8]:
x[torch.sort(x)[1]]

IndexError: index 2 is out of bounds for dimension 0 with size 1